In [ ]:
from google.colab import drive
import os
import tarfile
import json
import pickle
from tqdm import tqdm

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Specify paths
tar_file_path = 'drive/My Drive/evaluation/eval_all.tar'
local_extract_path = '/content/evaluation'
log_file_path = 'drive/My Drive/evaluation/extract_log.json'
members_file_path = 'drive/My Drive/evaluation/members.pkl'

# Step 3: Create directories if they don't exist
os.makedirs(local_extract_path, exist_ok=True)

# Step 4: Load progress log
if os.path.exists(log_file_path):
    with open(log_file_path, 'r') as log_file:
        progress_log = json.load(log_file)
else:
    progress_log = {"last_extracted_index": 0}

# Step 5: Check and load members list
if os.path.exists(members_file_path):
    with open(members_file_path, 'rb') as members_file:
        members = pickle.load(members_file)
    print("Loaded members list from members.pkl")
else:
    print("Generating members list from the tar file...")
    with tarfile.open(tar_file_path, 'r') as tar:
        members = tar.getmembers()
        members = [m for m in members if m.isfile()]  # Exclude directories
    with open(members_file_path, 'wb') as members_file:
        pickle.dump(members, members_file)
    print(f"Members list saved to {members_file_path}")

# Step 6: Extract the tar file with batch processing
batch_size = 10000
total_batches = len(members) // batch_size + (1 if len(members) % batch_size else 0)
start_index = progress_log["last_extracted_index"]

print("Starting extraction process...")
try:
    with tarfile.open(tar_file_path, 'r') as tar:
        for i in tqdm(range(start_index, len(members), batch_size),
                      initial=start_index // batch_size,
                      total=total_batches,
                      desc="Extracting files in batches"):
            batch = members[i:i + batch_size]

            for member in tqdm(batch, desc=f"Processing batch {i // batch_size + 1}", leave=False):
                try:
                    tar.extract(member, path=local_extract_path)
                    print(f"Extracted: {os.path.join(local_extract_path, member.name)}")
                except Exception as e:
                    print(f"Error extracting {member.name}: {e}")

            # Save progress after each batch
            progress_log["last_extracted_index"] = i + batch_size
            with open(log_file_path, 'w') as log_file:
                json.dump(progress_log, log_file)
            print(f"Batch {i // batch_size + 1} completed.")

except Exception as e:
    print(f"An error occurred during extraction: {e}")

print(f"Extraction process completed. Extracted files are available at: {local_extract_path}")

# Step 7: Verify extraction results
def count_files(directory):
    return sum(len(files) for _, _, files in os.walk(directory))

total_files = count_files(local_extract_path)
print(f"Total files extracted: {total_files}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded members list from members.pkl
Starting extraction process...


Extracting files in batches: 100%|██████████| 40/40 [00:00<?, ?it/s]

Extraction process completed. Extracted files are available at: /content/evaluation
Total files extracted: 0


In [ ]:
from google.colab import drive
import os
import zipfile
from tqdm import tqdm

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Specify paths
zip_file_path = 'drive/MyDrive/evaluation/extracted_files.zip'  # Path to the .zip file on Google Drive
extract_to_path = 'drive/MyDrive/evaluation/evaluation_extracted'               # Path to extract files locally

# Step 3: Create the extraction directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Step 4: Extract the .zip file
try:
    print(f"Extracting files from {zip_file_path} to {extract_to_path}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zipf:
        file_list = zipf.namelist()  # List of files in the .zip archive
        with tqdm(total=len(file_list), desc="Extracting files") as pbar:
            for file in file_list:
                zipf.extract(file, extract_to_path)
                pbar.update(1)  # Update the progress bar
    print(f"Extraction completed. Files are available at: {extract_to_path}")
except Exception as e:
    print(f"An error occurred during extraction: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracting files from drive/MyDrive/evaluation/extracted_files.zip to drive/MyDrive/evaluation/evaluation_extracted...


Extracting files: 0it [00:00, ?it/s]

Extraction completed. Files are available at: drive/MyDrive/evaluation/evaluation_extracted


In [ ]:
import os

def count_files_in_directory(directory_path):
    file_count = 0
    for root, dirs, files in os.walk(directory_path):
        file_count += len(files)  # Count the files in the current directory
    return file_count

# Example usage
directory_path = 'drive/My Drive/evaluation/evaluation_extracted'  # Change to your directory path
num_files = count_files_in_directory(directory_path)
print(f"Total number of files: {num_files}")


Total number of files: 0


In [ ]:
from google.colab import drive
import tarfile
import os
import json
import pickle
from tqdm import tqdm
import shutil

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Specify the paths
tar_file_path = 'drive/My Drive/evaluation/eval_all.tar'  # Path to the tar file
local_extract_path = '/content/evaluation'               # Temporary local extraction path
final_extract_path = 'drive/My Drive/evaluation'         # Final directory on Google Drive
log_file_path = 'drive/My Drive/evaluation/extract_log.json'  # Progress log file path
members_file_path = 'drive/My Drive/evaluation/members.pkl'  # Members list file path

# Step 3: Create directories if they don't exist
os.makedirs(local_extract_path, exist_ok=True)
os.makedirs(final_extract_path, exist_ok=True)

# Step 4: Load progress log
if os.path.exists(log_file_path):
    with open(log_file_path, 'r') as log_file:
        progress_log = json.load(log_file)
else:
    progress_log = {"last_extracted_index": 0}

# Step 5: Check and load members list
if os.path.exists(members_file_path):
    with open(members_file_path, 'rb') as members_file:
        members = pickle.load(members_file)
    print("Loaded members list from members.pkl")
else:
    print("Generating members list from the tar file...")
    with tarfile.open(tar_file_path, 'r') as tar:
        members = tar.getmembers()
    with open(members_file_path, 'wb') as members_file:
        pickle.dump(members, members_file)
    print(f"Members list saved to {members_file_path}")

# Step 6: Extract the tar file with batch processing
batch_size = 10000
total_batches = len(members) // batch_size + (1 if len(members) % batch_size else 0)

start_index = progress_log["last_extracted_index"]

print("Starting extraction process...")
try:
    with tarfile.open(tar_file_path, 'r') as tar:
        # Iterate over batches
        for i in tqdm(range(start_index, len(members), batch_size),
                      initial=start_index // batch_size,
                      total=total_batches,
                      desc="Extracting files in batches"):
            batch = members[i:i + batch_size]

            for member in tqdm(batch,
                               desc=f"Processing batch {i // batch_size + 1}",
                               leave=False):
                try:
                    tar.extract(member, path=local_extract_path)
                except Exception as e:
                    print(f"Error extracting {member.name}: {e}")
                    continue

            # Save progress after completing each batch
            progress_log["last_extracted_index"] = i + batch_size
            with open(log_file_path, 'w') as log_file:
                json.dump(progress_log, log_file)
            print(f"Batch {i // batch_size + 1} completed.")

    # Move extracted files to Google Drive
    print("Moving extracted files to Google Drive...")
    shutil.copytree(local_extract_path, final_extract_path, dirs_exist_ok=True)
    print(f"Files moved to: {final_extract_path}")

except Exception as e:
    print(f"An error occurred during extraction: {e}")

# Cleanup local extraction folder
if os.path.exists(local_extract_path):
    shutil.rmtree(local_extract_path)

print("Extraction process completed.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded members list from members.pkl
Starting extraction process...


Extracting files in batches:  38%|███▊      | 15/40 [00:22<09:12, 22.11s/it]

Batch 15 completed.



Extracting files in batches:  40%|████      | 16/40 [00:38<07:31, 18.83s/it]

Batch 16 completed.


...

Batch 38 completed.



Extracting files in batches:  98%|█████████▊| 39/40 [06:52<00:17, 17.12s/it]

Batch 39 completed.



Extracting files in batches: 100%|██████████| 40/40 [06:59<00:00, 16.15s/it]


Batch 40 completed.
Moving extracted files to Google Drive...


In [ ]:
import tarfile

tar_file_path = 'drive/My Drive/evaluation/eval_all.tar'

with tarfile.open(tar_file_path, 'r') as tar:
    members = tar.getmembers()
    print(f"Total members in tar file: {len(members)}")

    # Display details of the first 20 members
    for i, member in enumerate(members[:20]):
        print(f"Member {i+1}:")
        print(f"  Name: {member.name}")
        print(f"  Is File: {member.isfile()}")
        print(f"  Is Directory: {member.isdir()}")
        print(f"  Is Link: {member.islnk()}")
        print(f"  Size: {member.size} bytes")


Total members in tar file: 395610
Member 1:
  Name: eval_all
  Is File: False
  Is Directory: True
  Is Link: False
  Size: 0 bytes
Member 2:
  Name: eval_all/Eval_0000004.png
  Is File: True
  Is Directory: False
  Is Link: False
  Size: 74186 bytes
Member 3:
  Name: eval_all/Eval_0000005.png
  Is File: True
  Is Directory: False
  Is Link: False
  Size: 72377 bytes
Member 4:
  Name: eval_all/Eval_0000007.png
  Is File: True
  Is Directory: False
  Is Link: False
  Size: 86112 bytes
Member 5:
  Name: eval_all/Eval_0000008.png
  Is File: True
  Is Directory: False
  Is Link: False
  Size: 86709 bytes
Member 6:
  Name: eval_all/Eval_0000015.png
  Is File: True
  Is Directory: False
  Is Link: False
  Size: 74185 bytes
Member 7:
  Name: eval_all/Eval_0000016.png
  Is File: True
  Is Directory: False
  Is Link: False
  Size: 65312 bytes
Member 8:
  Name: eval_all/Eval_0000023.png
  Is File: True
  Is Directory: False
  Is Link: False
  Size: 91958 bytes
Member 9:
  Name: eval_all/Eval_000

In [ ]:
from google.colab import drive
import os
import tarfile
from tqdm import tqdm

# Step 2: Specify paths
tar_file_path = 'drive/My Drive/evaluation/eval_all.tar'
local_extract_path = '/content/evaluation'

# Step 3: Create directory for extraction
os.makedirs(local_extract_path, exist_ok=True)

# Step 4: Inspect members in the .tar file
print("Inspecting members in the .tar file...")
with tarfile.open(tar_file_path, 'r') as tar:
    members = tar.getmembers()
    print(f"Total members in tar file: {len(members)}")

    # Display details of the first 20 members
    for i, member in enumerate(members[:20]):
        print(f"Member {i+1}:")
        print(f"  Name: {member.name}")
        print(f"  Is File: {member.isfile()}")
        print(f"  Is Directory: {member.isdir()}")
        print(f"  Is Link: {member.islnk()}")
        print(f"  Size: {member.size} bytes")

# Step 5: Extract files
print("Starting extraction process...")
try:
    with tarfile.open(tar_file_path, 'r') as tar:
        for member in tqdm(members, desc="Extracting files"):
            if member.isfile():
                # Sanitize path if necessary
                member.name = os.path.basename(member.name)
                tar.extract(member, path=local_extract_path)
            else:
                print(f"Skipping: {member.name} (Not a file)")

    print(f"Extraction completed. Extracted files are available at: {local_extract_path}")
except Exception as e:
    print(f"An error occurred during extraction: {e}")

# Step 6: Verify extracted files
def list_files(directory, num_files=10):
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    print(f"Total files: {len(files)}")
    print("First few files:")
    print("\n".join(files[:num_files]))

list_files(local_extract_path)
